# Blockchain for School Marks

This notebook demonstrates the creation of a simple blockchain to store school marks. Each student's data is represented as a block in the blockchain.

In [ ]:
# Import necessary libraries
import hashlib as hasher  # For hashing block data
import datetime as date   # For timestamping blocks
import pandas as pd       # For handling student marks data
import os                 # For file operations

In [ ]:
# Load the CSV file containing marks data
# Provide a valid path or place the CSV in the same directory as this notebook
file_name = "Marks.csv"

if os.path.exists(file_name):
    df = pd.read_csv(file_name)  # Load CSV into a DataFrame
    print("Marks data loaded successfully!\n")
    print(df.head())  # Display the first few rows for confirmation
else:
    raise FileNotFoundError(f"The file {file_name} was not found. Ensure it is in the correct directory.")

In [ ]:
# Define the Block class
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        """
        Initialize a blockchain block.
        :param index: Position of the block in the chain
        :param timestamp: Time of block creation
        :param data: Block data (student marks)
        :param previous_hash: Hash of the previous block
        """
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
    
    def hash_block(self):
        """Generate a SHA-256 hash for the block."""
        sha = hasher.sha256()
        sha.update((str(self.index) + str(self.timestamp) +
                    str(self.data) + str(self.previous_hash)).encode('utf-8'))
        return sha.hexdigest()

# Blockchain utility functions
def create_genesis_block():
    """Create the first block in the blockchain."""
    return Block(0, date.datetime.now(), "Genesis Block", "0")

def next_block(last_block, block_data):
    """Create the next block in the blockchain."""
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, block_data, this_hash)

In [ ]:
# Create blockchain and add blocks
blockchain = [create_genesis_block()]  # Initialize with the genesis block
previous_block = blockchain[0]  # Reference to the last block

print("Adding blocks to the blockchain:\n")

# Add each row of marks data as a block
for i, row in df.iterrows():
    student_data = row.to_dict()  # Convert row to dictionary
    block_data = f"Student Record: {student_data}"  # Prepare block data
    block_to_add = next_block(previous_block, block_data)  # Create new block
    blockchain.append(block_to_add)  # Append to blockchain
    previous_block = block_to_add  # Update reference
    print(f"Block #{block_to_add.index} added with hash: {block_to_add.hash}")

In [ ]:
# Visualize the blockchain
# Display block details in a DataFrame
blockchain_data = [{
    "Index": block.index,
    "Timestamp": block.timestamp,
    "Data": block.data,
    "Previous Hash": block.previous_hash,
    "Hash": block.hash
} for block in blockchain]

# Convert to DataFrame and display
blockchain_df = pd.DataFrame(blockchain_data)
print("\nBlockchain Overview:")
blockchain_df

In [ ]:
def search_student_by_name(name):
    """
    Search for all entries of a student by name in the blockchain and display their marks.
    """
    found = False
    print(f"Searching for records of student: {name}\n" + "-"*50)
    for block in blockchain.chain:
        data = block.data
        if data.get("student_name", "").lower() == name.lower():
            found = True
            print(f"Block #{block.index} | Subject: {data.get('subject')} | Marks: {data.get('marks')} | Time: {block.timestamp}")
    if not found:
        print("No records found for this student.")

# Example usage
# search_student_by_name("Alice")

In [ ]:
# Load the CSV
df = pd.read_csv("Marks.csv")

# Check if required columns are there
required_columns = {"Student", "Subject", "Marks", "Exam"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"CSV file must contain the columns: {required_columns}")

# Calculate average marks per student
student_avg = df.groupby("Student")["Marks"].mean().reset_index()
student_avg.rename(columns={"Marks": "Average_Marks"}, inplace=True)

# Merge average with original data
df_with_avg = pd.merge(df, student_avg, on="Student")

# Compare current marks with student average
df_with_avg["Performance_Trend"] = df_with_avg.apply(
    lambda row: "Above Average" if row["Marks"] > row["Average_Marks"]
    else ("Below Average" if row["Marks"] < row["Average_Marks"] else "At Average"),
    axis=1
)

# Display updated dataframe
print("Student Performance vs Average:")
display(df_with_avg.sort_values(by=["Student", "Exam"]))
